In [ ]:
from sentinelhub import (
    SHConfig,
    DataCollection,
    SentinelHubCatalog,
    SentinelHubRequest,
    SentinelHubStatistical,
    BBox,
    bbox_to_dimensions,
    CRS,
    MimeType,
    Geometry,
)

from PIL import Image
import numpy as np
import os

import geopandas as gpd
from shapely.geometry import box

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from shapely.geometry import box
import cartopy
from utils import plot_image




In [ ]:
# Only run this cell if you have not created a configuration.

config = SHConfig()
config.sh_client_id = "sh-84d21c38-65f5-410d-9267-b1db4ba47b41"
config.sh_client_secret = "7ykprUj53bYD42W30gevmHFnI4U5y75M"
config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
config.sh_base_url = "https://sh.dataspace.copernicus.eu"
# config.save("cdse")

In [ ]:
catalog = SentinelHubCatalog(config=config)
resolution = 10

In [ ]:
def plot_aoi_box(aoi_coords_wgs84, show_bbox=True, bbox_color='red', map_title="AOI Map"):
    """
    Plots a map with an optional bounding box for a given AOI.

    Parameters:
    - aoi_coords_wgs84: List of [min_lon, min_lat, max_lon, max_lat] for the AOI
    - show_bbox: Boolean to toggle displaying the bounding box (default: True)
    - bbox_color: Color of the bounding box (default: 'red')
    - map_title: Title of the map (default: "AOI Map")
    """
    # Create figure and axis with PlateCarree projection
    fig, ax = plt.subplots(figsize=(10, 10), dpi=200, subplot_kw={'projection': ccrs.PlateCarree()})

    # Add map features (land, coastline, borders)
    ax.add_feature(cfeature.LAND, edgecolor='black')
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=":")

    # Manually set x and y limits to center the map on AOI
    ax.set_xlim(aoi_coords_wgs84[0] - 0.2, aoi_coords_wgs84[2] + 0.2)  # Expand longitude range slightly
    ax.set_ylim(aoi_coords_wgs84[1] - 0.2, aoi_coords_wgs84[3] + 0.2)  # Expand latitude range slightly

    # Create the bounding box as a Polygon
    bbox = box(aoi_coords_wgs84[0], aoi_coords_wgs84[1], aoi_coords_wgs84[2], aoi_coords_wgs84[3])

    # Optionally display the bounding box in red
    if show_bbox:
        ax.add_geometries([bbox], crs=ccrs.PlateCarree(), facecolor='none', edgecolor=bbox_color, linewidth=2)

    # Add a title
    plt.title(map_title, fontsize=14)

    # Show the plot
    plt.show()


In [ ]:
# Define AOI coordinates (min_lon, min_lat, max_lon, max_lat)
aoi_coords_wgs84 = [12.5, 44.7, 13.0, 45.1]

# Call the function to plot the map with the bounding box shown
plot_aoi_box(aoi_coords_wgs84, show_bbox=True, bbox_color='red', map_title="Po River Delta and Sea AOI")


In [ ]:


def download_sentinel_images(aoi_coords_wgs84, time_interval, output_dir="Images", resolution=10, max_size=2500):
    """
    Downloads Sentinel-2 true color images for a given AOI and time range.

    Parameters:
    - aoi_coords_wgs84: list of [min_lon, min_lat, max_lon, max_lat] (AOI bounding box)
    - time_interval: tuple of (start_date, end_date) in 'YYYY-MM-DD' format
    - output_dir: Directory to save the images (default: 'Po_Images')
    - resolution: Spatial resolution in meters per pixel (default: 10m)
    - max_size: Maximum image size (Sentinel Hub limit: 2500 pixels)
    """
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Define AOI Bounding Box
    aoi_bbox = BBox(bbox=aoi_coords_wgs84, crs=CRS.WGS84)
    aoi_size = bbox_to_dimensions(aoi_bbox, resolution=resolution)
    aoi_size = (min(aoi_size[0], max_size), min(aoi_size[1], max_size))  # Limit size

    # Search for available Sentinel-2 images
    search_iterator = catalog.search(
        DataCollection.SENTINEL2_L2A,
        bbox=aoi_bbox,
        time=time_interval,
        fields={"include": ["id", "properties.datetime"], "exclude": []},
    )

    results = list(search_iterator)
    print(f"Total number of results: {len(results)}")

    if not results:
        print("No images found for the given AOI and time interval.")
        return []

    # Define SentinelHub evalscript for true color imagery
    evalscript_true_color = """
        //VERSION=3
        function setup() {
            return { input: [{ bands: ["B02", "B03", "B04"] }], output: { bands: 3 } };
        }
        function evaluatePixel(sample) {
            return [sample.B04, sample.B03, sample.B02];
        }
    """

    true_color_imgs = []

    # Loop through results to fetch corresponding images
    for result in results:
        timestamp = result["properties"]["datetime"]
        print(f"Fetching image for timestamp: {timestamp}")

        # Request the image
        request_true_color = SentinelHubRequest(
            evalscript=evalscript_true_color,
            input_data=[
                SentinelHubRequest.input_data(
                    data_collection=DataCollection.SENTINEL2_L2A.define_from(
                        name="s2l2a", service_url="https://sh.dataspace.copernicus.eu"
                    ),
                    time_interval=(timestamp, timestamp),
                    other_args={"dataFilter": {"mosaickingOrder": "leastCC"}},
                )
            ],
            responses=[SentinelHubRequest.output_response("default", MimeType.PNG)],
            bbox=aoi_bbox,
            size=aoi_size,
            config=config,
        )

        # Get the image
        image_data = request_true_color.get_data()

        if image_data:
            image = image_data[0]  # Extract the first image
            true_color_imgs.append(image)

            # Save the image
            formatted_timestamp = timestamp[:19].replace(":", "-")
            filename = os.path.join(output_dir, f"sentinel_image_{formatted_timestamp}.png")
            img = Image.fromarray(np.uint8(image))
            img.save(filename)

            print(f"Saved: {filename}")

    print(f"Total images downloaded: {len(true_color_imgs)}")
    return true_color_imgs  # Return the list of images

# Example usage
aoi_coords_wgs84 = [12.5, 44.7, 13.0, 45.1]  # Example coordinates for Po River Delta
time_interval = ("2024-03-01", "2024-03-10")  # Example time range

download_sentinel_images(aoi_coords_wgs84, time_interval)


In [ ]:
# Nord Corsica

In [ ]:
# Define AOI coordinates (min_lon, min_lat, max_lon, max_lat)
aoi_coords_wgs84 = [9.288940,43.012681,9.481888,43.119530] 

# Call the function to plot the map with the bounding box shown
plot_aoi_box(aoi_coords_wgs84, show_bbox=True, bbox_color='red', map_title="Nord Corsica AOI")

In [ ]:
aoi_coords_wgs84 = [9.288940,43.012681,9.481888,43.119530] 
time_interval = ("2023-03-01", "2023-03-10")  # Example time range

download_sentinel_images(aoi_coords_wgs84, time_interval)

In [ ]:
# gulf of Genova

In [ ]:
# Define AOI coordinates (min_lon, min_lat, max_lon, max_lat)
aoi_coords_wgs84 = [8.322144,43.271206,9.687195,44.140827]
# Call the function to plot the map with the bounding box shown
plot_aoi_box(aoi_coords_wgs84, show_bbox=True, bbox_color='red', map_title="Gulf of Genova")

In [ ]:
AOI = 'Gulf of Genova'
aoi_coords_wgs84 = [8.322144,43.271206,9.687195,44.140827] 
time_interval = ("2023-03-01", "2023-03-10")  # Example time range

download_sentinel_images(aoi_coords_wgs84, time_interval)